##### Silver Speech and Golden Silence: Spoiler Detection Project##### Spoiler Alert! Spoiler Detection Project

### Sentence-wise SGD with Metadata (combined in 1 feature)

In [18]:
#Imports
import pandas as pd
import numpy as np
from tqdm import tqdm
from imblearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.feature_selection import SelectKBest, chi2
from sklearn import model_selection
from sklearn.utils import class_weight
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.linear_model import SGDClassifier

#### Load Data

In [3]:
train = pd.read_json('/Users/juliaschafer/NF_Capstone_Spoiler_Detection/data/train_reduced.json')
train.drop(['user_id', 'title'], inplace = True, axis = 1)
#Correct the review_length column
train['review_len'] = train['tokenized'].apply(lambda x: len(x))

In [4]:
val = pd.read_json('/Users/juliaschafer/NF_Capstone_Spoiler_Detection/data/validation_reduced.json')
val.drop(['user_id', 'title'], inplace = True, axis = 1)
#Correct the review_length column
val['review_len'] = val['tokenized'].apply(lambda x: len(x))

In [5]:
#Import reviews as string (not in list)
X_tr = pd.read_csv('/Users/juliaschafer/NF_Capstone_Spoiler_Detection/data/X_train.csv')
X_v = pd.read_csv('/Users/juliaschafer/NF_Capstone_Spoiler_Detection/data/X_val.csv')
X_tr.rename(columns = {"0": 'review_whole'}, inplace = True)
X_v.rename(columns = {"0": 'review_whole'}, inplace = True)

In [6]:
#Combine data 
train = pd.concat([train, X_tr.review_whole], axis = 1)
val = pd.concat([val, X_v.review_whole], axis = 1)

In [8]:
train.to_json('/Users/juliaschafer/NF_Capstone_Spoiler_Detection/data/train_only_needed_columns.json')
val.to_json('/Users/juliaschafer/NF_Capstone_Spoiler_Detection/data/val_only_needed_columns.json')

In [9]:
#Since the data is very inbalanced, we only take the training reviews containing spoiler sentences. 
train = train[train['spoiler_dum']== 1]

In [10]:
train.reset_index(inplace = True, drop = True)

In [11]:
#Function to split the reviews into single sentences
def get_sentencewise_df(df):
    
    '''Get review sentences with according label, genre, review length from a dataframe df given in.
    The review sentences are written in a list 'lst' which is then transformed
    into a numpy array'''
    
    sen_lst = []
    genre_lst = []
    len_lst = []
    label_lst = []
    for i in range(len(df)):
        for sentence in df['tokenized'][i]:
            sen_lst.append(sentence)
            genre_lst.append(df['genre'][i])
            len_lst.append(df['review_len'][i])
            
    for labellist in df['sentence_labels']:
        for label in labellist:
            label_lst.append(label)
    df = pd.concat([pd.Series(sen_lst), pd.Series(label_lst), pd.Series(genre_lst), pd.Series(len_lst)], axis = 1)
    return df

In [12]:
#Train data sentence-wise
train = get_sentencewise_df(train)
train.rename(columns = {0: 'sentence', 1: 'spoiler_dum', 2: 'genre', 3: 'review_length'}, inplace = True)
train.head()

,sentence,spoiler_dum,genre,review_length
0,overall plot book get star maybe,0,"fantasy, paranormal",11
1,bad trilogy end,0,"fantasy, paranormal",11
2,feeling chase character aside basically feelin...,1,"fantasy, paranormal",11
3,unacceptable,1,"fantasy, paranormal",11
4,give book star go able read case later book se...,0,"fantasy, paranormal",11


In [13]:
#Validation data sentence-wise
val = get_sentencewise_df(val)
val.rename(columns = {0: 'sentence', 1: 'spoiler_dum', 2: 'genre', 3: 'review_length'}, inplace = True)
val.head()

,sentence,spoiler_dum,genre,review_length
0,look like promising new fantasy series,0,"fantasy, paranormal",3
1,love character definitely omg moment read,0,"fantasy, paranormal",3
2,negative wish term mythology explain little well,0,"fantasy, paranormal",3
3,definitely remind inuyasha read,0,"fantasy, paranormal",7
4,blend humour romance delightful,0,"fantasy, paranormal",7


In [14]:
#New column with genre and review text
train[['text_and_genre']] = train.genre + ' ' + train.sentence
val[['text_and_genre']] = val.genre + ' ' + val.sentence

In [15]:
X_train = train[['sentence', 'genre', 'review_length']]
X_train_rev = train.text_and_genre
y_train = train.spoiler_dum

In [16]:
X_val = val[['sentence', 'genre', 'review_length']]
X_val_rev = val.text_and_genre
y_val = val.spoiler_dum

#### Balanced Random Forest 

In [19]:
#Build a pipeline for feature extraction with TF IDF and SGD
#CountVectorizer
vectorizer = CountVectorizer(stop_words = 'english', ngram_range = (1,2), min_df = 100)

#Feature Selection: K-Best
feat = SelectKBest(score_func = chi2, k = 5000)

#Balanced Random Forest classifier
sgd = SGDClassifier(random_state = 42, penalty = 'elasticnet', alpha = .001, class_weight = {0: 0.3, 1: 0.7}, 
                    l1_ratio = 0, max_iter = 1000, loss = 'perceptron', shuffle = True, n_jobs = -1)

pipe = Pipeline([('count', vectorizer),('select_kbest', feat), ('SGD', sgd)])

In [20]:
#Function for model training and prediction
def run_model(pipeline, X_train, y_train, X_test, y_test):
    #Fit the model
    model = pipeline.fit(X_train, y_train)
    
    #Predict labels of test data
    y_pred = model.predict(X_test)
    
    #Print classification report and confusion matrix
    return print(classification_report(y_test, y_pred)), print(confusion_matrix(y_test, y_pred, normalize = 'true'))
    return y_pred

In [21]:
#Run the model sentence-wise (validation data)
run_model(pipe, X_train_rev, y_train, X_val_rev, y_val)

              precision    recall  f1-score   support

           0       0.98      0.52      0.68   3396530
           1       0.05      0.72      0.09    113966

    accuracy                           0.52   3510496
   macro avg       0.51      0.62      0.38   3510496
weighted avg       0.95      0.52      0.66   3510496

[[0.51532122 0.48467878]
 [0.27738975 0.72261025]]


(None, None)